## Scrapping de congresovisible.org la Asistencia de Senadores en Feb~May 2018
Usando las votaciones (voto + abstencion) como proxy de su asistencia

In [2]:
import re
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [3]:
import unicodedata

def elimina_tildes(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

In [10]:
def GetListSenators(soup):
    
    names = []

    ## Votantes
    table = soup.find('table', attrs={'id':'tabla-reporte-detallado'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')

    for row in rows:
        name = row.find('td').get_text()
        names.append(name)
    
    ## Abstenciones
    table = soup.find('table', attrs={'id':'tabla-reporte-abstenciones'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')

    for row in rows:
        name = row.find('td').get_text()
        names.append(name)

    ## Eliminar tildes
    names = [elimina_tildes(name) for name in names ]
    print(names)
    return names

In [38]:
def GetDate(soup):
    panel = soup.find_all("div", attrs={"class": "module-contenido"})

    ## Quick and dirty direct way
    # print(panel[1].contents[1].contents[7].find('p').getText()) 

    for pan in panel:
        for pa in pan.find_all('li'):
            text = pa.find('p').get_text()
            if "201" in text:
                text = text.replace('d ','')
                text = text.replace('de ','')
                print(text)
                return text

In [39]:
page = requests.get("https://www.congresovisible.org/votaciones/%d" % 13960)
print(page.status_code)
soup = BeautifulSoup(page.content, 'html.parser')
date = GetDate(soup)
senators = GetListSenators(soup)

200
Nov 30 2016
['Thania Vega de Plazas', 'German Varon Cotrino', 'Paloma Susana Valencia Laserna', 'Alvaro Uribe Velez', 'Nohora Stella Tovar Rey ', 'Carlos Enrique Soto Jaramillo', 'Guillermo Antonio Santos Marin', 'Milton Arlex Rodriguez Sarmiento', 'Roosvelt Rodriguez Rengifo', 'Juan Carlos Restrepo Escobar', 'Alfredo Rangel Suarez', 'Alfredo Ramos Maya ', 'Jorge Elieser Prieto Riveros', 'Jorge Hernando Pedraza Gutierrez', 'Myriam Alicia Paredes Aguirre', 'Jorge Ivan Ospina Gomez ', 'Antonio Jose Navarro Wolff ', 'Ivan Leonidas Name Vasquez', 'Viviane Aleyda Morales Hoyos', 'Manuel Guillermo Mora Jaramillo', 'Juan Samy Merheg Marun', 'Carlos Felipe Mejia Mejia ', 'Rosmery Martinez Rosales ', 'Maritza Martinez Aristizabal', 'Ernesto Macias Tovar ', 'Claudia Nayibe Lopez Hernandez ', 'Oscar Mauricio Lizcano Arango', 'German Dario Hoyos Giraldo', 'Julio Miguel Guerra Sotto ', 'Maria del Rosario Guerra de la Espriella ', 'Antonio del Cristo Guerra de la Espriella', 'Juan Diego Gomez Ji

In [24]:
votaciones = np.arange(13986,13992,1)
for votacion in votaciones:
    print("Votacion %d" % votacion)
    page = requests.get("https://www.congresovisible.org/votaciones/%d" % votacion)
    
    if page.status_code == 404:
        print("Status code 404. Skipping....")
        continue
    
    soup = BeautifulSoup(page.content, 'html.parser')
    date = GetDate(soup)
#     senators = GetListSenators(soup)

Votacion 13986
[<div class="module-contenido"><div class="container-table4"><table class="table4 table4-1 table4-1-1"><thead><tr><th class="estilo1">No</th><th class="estilo2">Si</th><!--                                                        <th><div class="contenedor-vinculos6"><a title="Ver más " class="vinculo-interrogacion" href="#">Más información</a></div></th>--></tr></thead><tbody><tr><td class="estilo1"><span class="estilo3">0%<span class="numero-voto">(0)</span></span><div class="grafica1 grafica1-desacuerdo"><div class="item-grafica" style="width: 0%;"></div></div></td></tr></tbody></table></div></div>, <div class="module-contenido"><h2>Datos generales</h2><ul class="lista6"><!--                                            <li>--><!--                                                <h3>Legislatura</h3>--><!--                                                <p>2001-2010 1 periodo</p>--><!--                                            </li>--><li><h3>Cámara</h3><p>Cámara de Repre

In [156]:
df = pd.DataFrame({'senator':senators, 'date':date})
df = df.set_index('senator')
df
# dummy_sen = pd.get_dummies(df["senator"], prefix="is_")
# dummy_sen

,date
senator,
Berner Leon Zambrano Eraso,May 29 2018
Paloma Susana Valencia Laserna,May 29 2018
Oscar Hernan Sanchez Leon,May 29 2018
Miguel Angel Pinto Hernandez,May 29 2018
Samuel Alejandro Hoyos Mejia,May 29 2018
Juan Carlos Garcia Gomez,May 29 2018
Alejandro Carlos Chacon Camargo,May 29 2018
Jose Edilberto Caicedo Sastoque,May 29 2018
Maria Fernanda Cabal Molina,May 29 2018


In [157]:
df.T

senator,Berner Leon Zambrano Eraso,Paloma Susana Valencia Laserna,Oscar Hernan Sanchez Leon,Miguel Angel Pinto Hernandez,Samuel Alejandro Hoyos Mejia,Juan Carlos Garcia Gomez,Alejandro Carlos Chacon Camargo,Jose Edilberto Caicedo Sastoque,Maria Fernanda Cabal Molina,Carlos Alberto Baena Lopez,...,Jose Obdulio Gaviria Velez,Heriberto Sanabria Astudillo,Hernan Francisco Andrade Serrano,Humphrey Roa Sarmiento,Juan Manuel Galan Pachon,Julian Bedoya Pulgarin,Andres Felipe Villamizar Ortiz,Fernando de la Pena Marquez,Jaime Buenahora Febres,Albeiro Vanegas Osorio
date,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,...,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018,May 29 2018
